## 아파트 밀집도

In [117]:
import pandas as pd
import numpy as np
import json
import requests
from tqdm import tqdm

data_seungnam = pd.read_csv("/Users/shinjh/Desktop/likelion/mid_project/dataset_교통접근성/경기도 성남시_공동주택현황_20240705.csv", encoding='cp949')
data_goyang = pd.read_csv("/Users/shinjh/Desktop/likelion/mid_project/dataset_교통접근성/경기도 고양시_공동주택현황_20231231.csv", encoding='cp949')

In [58]:
data_seungnam_sort = data_seungnam[data_seungnam['구']=='분당구']
data_seungnam_dong = [ "야탑동", "이매동", "서현동", "분당동", "수내동", "정자동", "금곡동", "구미동"]
data_seungnam_sort = data_seungnam_sort[data_seungnam_sort['동'].isin(data_seungnam_dong)]
# 아파트는 5층 이상 건물을 지칭
data_bundang = data_seungnam_sort[data_seungnam_sort['지상층(고)']>=5].reset_index(drop=True)
print(data_bundang.shape)
display(data_bundang.head())


(129, 10)


,구,동,단지명,지번주소,도로명주소,지상층(고),동수,세대수,관리사무소,데이터기준일자
0,분당구,구미동,까치마을4단지롯데선경,성남시 분당구 구미동 63,"경기도 성남시 분당구 미금일로 58 (구미동, 까치마을)",22.0,16,1124,031-715-6697,2024-07-05
1,분당구,구미동,까치마을신원,성남시 분당구 구미동 66,"경기도 성남시 분당구 미금로 177 (구미동, 까치마을)",25.0,14,882,031-715-3339,2024-07-05
2,분당구,구미동,까치마을1단지대우,성남시 분당구 구미동 77,"경기도 성남시 분당구 미금로 184 (구미동, 까치마을)",25.0,16,976,031-715-6278,2024-07-05
3,분당구,구미동,까치마을주공2단지,성남시 분당구 구미동 88,"경기도 성남시 분당구 미금일로 22 (구미동, 까치마을)",25.0,9,768,031-715-0403,2024-07-05
4,분당구,구미동,하얀마을 주공5단지,성남시 분당구 구미동 110,"경기도 성남시 분당구 미금일로 21 (구미동, 하얀마을)",21.0,7,779,031-717-3403,2024-07-05


In [59]:
data_goyang_sort = data_goyang[data_goyang['지번주소'].str.contains('일산')]
data_ilsan = data_goyang_sort[data_goyang_sort['건물유형']=='아파트'].reset_index(drop=True)
print(data_ilsan.shape)
display(data_ilsan.head())

(220, 15)


,순번,구분,단지명,지번주소,사업승인일,사용검사일,건물유형,층수,동수,세대수,관리소전화,팩스,연면적(제곱미터),승강기,비고
0,48,비의무,세인아파트,경기도 고양시 일산서구 일산동 587-5,1985-04-12,1985-12-13,아파트,5,2,70,NaN,NaN,5120.0,0,NaN
1,69,비의무,삼남아파트,경기도 고양시 일산서구 일산동 587-1,1985-12-02,1986-09-05,아파트,5,2,95,NaN,NaN,6927.0,0,NaN
2,82,비의무,신영아파트,경기도 고양시 일산서구 일산동 1818-0,1986-05-13,1987-02-19,아파트,5,2,120,NaN,NaN,7296.0,0,NaN
3,89,비의무,평일아파트,경기도 고양시 일산서구 일산동 586-21,1986-02-19,1987-11-19,아파트,4,2,48,NaN,NaN,2944.0,0,NaN
4,106,비의무,철산아파트,경기도 고양시 일산서구 덕이동 277-10,1988-03-17,1989-03-29,아파트,5,6,270,NaN,NaN,14685.0,0,NaN


## 데이터 전처리 (위경도추가))

In [130]:
import re

# 데이터 추가 준비
data_bundang_latlon = data_bundang.copy()[['단지명', '지번주소']]
conflict_idx = data_bundang_latlon[data_bundang_latlon['지번주소'].str.contains(",")].index
# 예외사항 제거 (반점)
if len(conflict_idx) > 0:
    data_bundang_latlon.loc[conflict_idx, '지번주소'] = data_bundang_latlon.loc[conflict_idx, '지번주소'].str.split(",").str[0]

data_bundang_latlon['위도'] = ""
data_bundang_latlon['경도'] = ""

#-------------------------
# 정규표현식 적용 함수
def clean_address(address):
    return re.sub(r"\s*외\s*\d+필지$", "", address)

data_ilsan_latlon = data_ilsan.copy()[['단지명', '지번주소']]
# 예외사항 제거 (반점)

conflict_idx = data_ilsan_latlon[data_ilsan_latlon['지번주소'].str.contains(",")].index
if len(conflict_idx) > 0:
    data_ilsan_latlon.loc[conflict_idx, '지번주소'] = data_ilsan_latlon.loc[conflict_idx, '지번주소'].str.split(",").str[0]

data_ilsan_latlon['지번주소'] = data_ilsan_latlon['지번주소'].apply(clean_address)
    
data_ilsan_latlon['위도'] = ""
data_ilsan_latlon['경도'] = ""


In [131]:
API_KEY = "20953ebae869efd8abdc3657eba800b6"

def get_location_kakao(address_data):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address_data
  headers = {"Authorization": f"KakaoAK {API_KEY}"}
  api_json = json.loads(str(requests.get(url,headers=headers).text))
  try:
      address = api_json['documents'][0]['address']
      crd = f"{str(address['y'])}, {str(address['x'])}"
  except:
      print(f"Can't Get data by some reason : {address_data}")
      crd = None
      return None, None
  return address, crd

In [132]:
def location_processing(df):
    for idx, i in enumerate(tqdm(df['지번주소'])):
        _, crd = get_location_kakao(i)

        if crd != None:
            lat, lon = crd.split(",")
        else:
            lat, lon = None, None
        
                    
        df.loc[idx, '위도'] = lat
        df.loc[idx, '경도'] = lon
        
    return df

In [149]:
data_bundang_latlon = location_processing(data_bundang_latlon)

100%|██████████| 129/129 [00:08<00:00, 15.30it/s]


In [150]:
data_bundang_latlon

,단지명,지번주소,위도,경도
0,까치마을4단지롯데선경,성남시 분당구 구미동 63,37.3468815575605,127.113362002443
1,까치마을신원,성남시 분당구 구미동 66,37.3493033801965,127.114166520456
2,까치마을1단지대우,성남시 분당구 구미동 77,37.3501363725723,127.117124715749
3,까치마을주공2단지,성남시 분당구 구미동 88,37.3475578274914,127.117287518118
4,하얀마을 주공5단지,성남시 분당구 구미동 110,37.3456085036719,127.117234490172
...,...,...,...,...
124,"동양정자파라곤 1단지(101, 102동)",성남시 분당구 정자동 14-3,37.3689682387682,127.106503245937
125,"동양정자파라곤 2단지(103, 104동)",성남시 분당구 정자동 14-1,37.3696434706224,127.106499684699
126,"동양정자파라곤 3단지(105, 106동)",성남시 분당구 정자동 13-1,37.3705645027605,127.10649545481
127,성원 상떼뷰리젠시 (201동),성남시 분당구 정자동 12-2,37.3703089701728,127.105592952815


In [133]:
data_ilsan_latlon = location_processing(data_ilsan_latlon)

 99%|█████████▊| 217/220 [00:15<00:00, 14.77it/s]

Can't Get data by some reason : 경기도 고양시 일산서구 일산동 1945


100%|██████████| 220/220 [00:15<00:00, 14.34it/s]

Can't Get data by some reason : 경기도 고양시 일산동구 식사동 식사2구역 A2블록


In [146]:
# 일부 정보 구글맵 통한 수기 입력
data_ilsan_latlon.loc[214, '위도'] = 37.690569
data_ilsan_latlon.loc[214, '경도'] = 126.773105

data_ilsan_latlon.loc[219, '위도'] = 37.672308
data_ilsan_latlon.loc[219, '경도'] = 126.811186

In [147]:
data_ilsan_latlon

,단지명,지번주소,위도,경도
0,세인아파트,경기도 고양시 일산서구 일산동 587-5,37.6877625549917,126.774284527201
1,삼남아파트,경기도 고양시 일산서구 일산동 587-1,37.6885336845352,126.773806241602
2,신영아파트,경기도 고양시 일산서구 일산동 1818-0,37.6896081683704,126.774454667414
3,평일아파트,경기도 고양시 일산서구 일산동 586-21,37.6879968857117,126.773619670675
4,철산아파트,경기도 고양시 일산서구 덕이동 277-10,37.6952068798832,126.758732043069
...,...,...,...,...
215,일산 센트럴 아이파크(현대산업개발),경기도 고양시 일산동구 중산동 1841,37.6788901587183,126.785219061471
216,일산에듀포레푸르지오(대우),경기도 고양시 일산서구 탄현동 1650,37.7040179844867,126.76799718347
217,일산2차아이파크,경기도 고양시 일산동구 중산동 1842,37.6778775017609,126.783467865681
218,일산자이2차,경기도 고양시 일산동구 식사동 852,37.6739602488977,126.808700389491


In [151]:
data_bundang_latlon.to_csv("/Users/shinjh/Desktop/likelion/mid_project/dataset_교통접근성/dataset_apt_list_2024_bundang.csv", encoding='utf-8')
# data_ilsan_latlon.to_csv("/Users/shinjh/Desktop/likelion/mid_project/dataset_교통접근성/dataset_apt_list_2024_ilsan.csv", encoding='utf-8')